# PIA Sistemas Adaptativos Martes M1
# Los Pros

# Carga de librerias y Datos



**Importar librerias**

In [ ]:
import os
import datetime
from datetime import date
import math
import IPython
import IPython.display
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from numpy import array
from pandas import to_datetime


**Importar los Datasets** 

In [ ]:
url_conf = 'https://raw.githubusercontent.com/eduardosalaz/Covid19-NuevoLeon/master/confimados.csv'
df_confirmados = pd.read_csv(url_conf, header=0)
print('Estatal: ')
print(df_confirmados.head())
print(df_confirmados.tail())
url_municipios = 'https://raw.githubusercontent.com/eduardosalaz/Covid19-NuevoLeon/master/municipios.csv'
df_municipios = pd.read_csv(url_municipios)
print('\nMunicipal: ')
print(df_municipios.head())
print(df_municipios.tail())
# define input sequence
df_confirmados.columns = ['ds','y']
casos = df_confirmados['y']
casos = casos.tolist()



Estatal: 
        fecha  total
0  2020-04-11    192
1  2020-04-12    201
2  2020-04-13    208
3  2020-04-14    216
4  2020-04-15    230
          fecha  total
215  2020-11-12  89756
216  2020-11-13  90427
217  2020-11-14  91115
218  2020-11-15  91804
219  2020-11-16  92483

Municipal: 
        fecha  monterrey  guadalupe  ...  melchorocampo  rayones  otrosEstados
0  2020-04-11         56         25  ...              0        0             8
1  2020-04-12         59         26  ...              0        0             8
2  2020-04-13         60         26  ...              0        0             9
3  2020-04-14         61         29  ...              0        0             9
4  2020-04-15         69         29  ...              0        0             9

[5 rows x 53 columns]
          fecha  monterrey  guadalupe  ...  melchorocampo  rayones  otrosEstados
215  2020-11-12      25242      12072  ...             12        7           683
216  2020-11-13      25421      12143  ...            

# Trabajar a nivel Estatal

**obtener la fecha a predecir**

In [ ]:
#@title Seleccionar Fecha a Predecir
fecha_predecir = "2021-08-12" #@param {type:"date"}

df_confirmados.columns = ['ds','y']
df_confirmados['ds'] = to_datetime(df_confirmados['ds'])

fecha_predecir = datetime.datetime.strptime(fecha_predecir, '%Y-%m-%d')

ultimo = df_confirmados.tail(1)
ultima_fecha_str = ultimo.iloc[0]['ds']
ultima_fecha = ultima_fecha_str.to_pydatetime()
delta = fecha_predecir - ultima_fecha
delta_days = delta.days

 **Separar el Dataset en Datasets de entrenamiento y prueba**


In [ ]:

#def split_sequence(sequence, n_steps):
	#return array(X), array(y)

def separarData(casos):

	# choose a number of time steps
	n_steps = 5
	# split into samples
	#X, y = split_sequence(casos, n_steps)
	X, y = list(), list()
	sequence=casos
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)	
	X = array(X)
	y = array(y)
	# reshape from [samples, timesteps] into [samples, timesteps, features]
	n_features = 1
	X = X.reshape((X.shape[0], X.shape[1], n_features))
	return X,y,n_steps,n_features
	
X,y,n_steps,n_features = separarData(casos)
# summarize the data
print('entrenamiento y prueba')
for i in range(3):
	print(X[i], y[i])

entrenamiento y prueba
[[192]
 [201]
 [208]
 [216]
 [230]] 241
[[201]
 [208]
 [216]
 [230]
 [241]] 246
[[208]
 [216]
 [230]
 [241]
 [246]] 258


**Definir el modelo de la LSTM**

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu',return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

**Entrenar la LSTM**

In [ ]:
def entrenarLSTM(X,y):
  model.fit(X, y, epochs=200, verbose=0)


**Mostrar Predicciones**


In [ ]:
def predecir(X,y,model,casos):
  x_input= casos[-5:]
  x_input = array(x_input)
  x_input = x_input.reshape((1, n_steps, n_features))
  yhat = model.predict(x_input, verbose=0)
  yhat = math.floor(yhat[0][0])
  casos.append(yhat)

for i in range(delta_days):

  separarData(casos)
  entrenarLSTM(X,y)
  predecir(X,y,model,casos)

prediccion = casos[-1]
casos = df_confirmados['y']
casos = casos.tolist()  


In [ ]:
print("El valor de la predicción para: " + str(fecha_predecir) + " es: " + str(prediccion) + " casos")

El valor de la predicción para: 2021-08-12 00:00:00 es: 468555 casos


# Trabajar a nivel municipal

**Seleccionar municipio**

In [ ]:
#@title Seleccionar Municipio para predecir
municipio_name_str = "Allende" #@param ["Abasolo", "Agualeguas", "Allende", "Anahuac", "Apodaca", "Aramberri", "Bustamante", "Cadereyta", "El Carmen", "Cerralvo", "China", "Cienega de Flores", "Doctor Arroyo", "Doctor Coss", "Doctor Gonzalez", "Galeana", "Garcia", "General Bravo", "General Escobedo", "General Teran", "General Trevino", "General Zaragoza", "General Zuazua", "Guadalupe", "Hidalgo", "Higueras", "Hualahuises", "Iturbide", "Juarez", "Lampazos", "Linares", "Los Aldamas", "Los Herreras", "Los Ramones", "Marin", "Melchor Ocampo", "Mier y Noriega", "Mina", "Montemorelos", "Monterrey", "Paras", "Pesqueria", "Rayones", "Sabinas Hidalgo", "Salinas Victoria", "San Nicolas de los Garza", "San Pedro", "Santa Catarina", "Santiago", "Vallecillo", "Villaldama"]
lista_df = list(df_municipios.columns)
copia_name = municipio_name_str
municipio_name_str = municipio_name_str.lower()
import re
sentence = re.sub(r"\s+", "", municipio_name_str, flags=re.UNICODE)


pattern = re.compile(sentence)

diccionario = {i:df_municipios.columns.get_loc(i) for i in df_municipios.columns}
dict_solo = {}

for buscar in diccionario.keys():
  if pattern.search(buscar):
    nombre = buscar
    numero_col = diccionario[buscar]

df_especifico = df_municipios[['fecha', nombre]].copy()

# define input sequence
df_especifico.columns = ['ds','y']
casosM = df_especifico['y']
casosM = casosM.tolist()
print(df_especifico)

             ds    y
0    2020-04-11    0
1    2020-04-12    0
2    2020-04-13    0
3    2020-04-14    0
4    2020-04-15    0
..          ...  ...
215  2020-11-12  697
216  2020-11-13  702
217  2020-11-14  702
218  2020-11-15  710
219  2020-11-16  717

[220 rows x 2 columns]


**Seleccionar Fecha a predecir**

In [ ]:
#@title Seleccionar Fecha a Predecir
fecha_predecir = "2020-11-29" #@param {type:"date"}
fecha_predecir = datetime.datetime.strptime(fecha_predecir, '%Y-%m-%d')
df_especifico.columns = ['ds','y']
df_especifico['ds'] = to_datetime(df_especifico['ds'])
ultimo = df_especifico.tail(1)
ultima_fecha_str = ultimo.iloc[0]['ds']
ultima_fecha = ultima_fecha_str.to_pydatetime()
delta = fecha_predecir - ultima_fecha
delta_days = delta.days


**Separar el dataset en entrenamiento y prueba**

In [ ]:

def separarDataM(casosM):

	# choose a number of time steps
	n_stepsM = 5
	# split into samples
	#X, y = split_sequence(casos, n_steps)
	XM, yM = list(), list()
	sequenceM=casosM
	for i in range(len(sequenceM)):
		# find the end of this pattern
		end_ixM = i + n_stepsM
		# check if we are beyond the sequence
		if end_ixM > len(sequenceM)-1:
			break
		# gather input and output parts of the pattern
		seq_xM, seq_yM = sequenceM[i:end_ixM], sequenceM[end_ixM]
		XM.append(seq_xM)
		yM.append(seq_yM)	
	XM = array(XM)
	yM = array(yM)
	# reshape from [samples, timesteps] into [samples, timesteps, features]
	n_featuresM = 1
	XM = XM.reshape((XM.shape[0], XM.shape[1], n_featuresM))
	return XM,yM,n_stepsM,n_featuresM
	
XM,yM,n_stepsM,n_featuresM = separarDataM(casosM)
# summarize the data
for i in range(3):
	print(XM[i], yM[i])

[[0]
 [0]
 [0]
 [0]
 [0]] 0
[[0]
 [0]
 [0]
 [0]
 [0]] 0
[[0]
 [0]
 [0]
 [0]
 [0]] 0


**Definir el modelo de la LSTM**

In [ ]:
modelM = Sequential()
modelM.add(LSTM(50, activation='relu',return_sequences=True, input_shape=(n_stepsM, n_featuresM)))
modelM.add(LSTM(50, activation='relu'))
modelM.add(Dense(1))
modelM.compile(optimizer='adam', loss='mse')

**Entrenar la LSTM**

In [ ]:
def entrenarLSTMM(XM,yM):
  modelM.fit(XM, yM, epochs=200, verbose=0)

**Mostrar Predicciones**

In [ ]:
def predecirM(XM,yM,modelM,casosM):
  x_input= casosM[-5:]
  x_input = array(x_input)
  x_input = x_input.reshape((1, n_stepsM, n_featuresM))
  yhat = modelM.predict(x_input, verbose=0)
  yhat = math.floor(yhat[0][0])
  casosM.append(yhat)

for i in range(delta_days):

  separarDataM(casosM)
  entrenarLSTMM(XM,yM)
  predecirM(XM,yM,modelM,casosM)

prediccion = casosM[-1]
casosM = df_especifico['y']
casosM = casosM.tolist()  


In [ ]:
print("El valor de la predicción para: " + str(fecha_predecir) + " es: " + str(prediccion) + " casos")


El valor de la predicción para: 2020-11-29 00:00:00 es: 756 casos


# Referencias



*   [Tensor Flow Time Series Forecasting](https://www.tensorflow.org/tutorials/structured_data/time_series)
*   [Machine Learning Mastery by Jason Brownlee](https://machinelearningmastery.com/)

